In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

In [ ]:
os.listdir("/kaggle/input/kdd-cup-1999-data")

In [ ]:
!cat /kaggle/input/kdd-cup-1999-data/kddcup.names

In [ ]:
col_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 
             'dst_bytes','land', 'wrong_fragment', 'urgent', 'hot', 
             'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
             'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 
             'num_access_files', 'num_outbound_cmds', 'is_host_login', 
             'is_guest_login', 'count', 'srv_count', 'serror_rate', 
             'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 
             'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 
             'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 
             'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
             'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 
             'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 
             'dst_host_srv_rerror_rate', 'label']

In [ ]:
df = pd.read_csv('/kaggle/input/kdd-cup-1999-data/kddcup.data.gz', names=col_names)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df['label'].value_counts()

In [ ]:
df['label'].nunique()

In [ ]:
num_cols = df._get_numeric_data().columns

cate_cols = list(set(df.columns)-set(num_cols))

cate_cols

In [ ]:
df[cate_cols]

In [ ]:
# Make a copy of the initial dataframe so we can use different approach later
init_df = df.copy(deep=True)

In [ ]:
for category in cate_cols:
    print(df[category].value_counts())

In [ ]:
category_mapping = {}
# Convert the non numerical values into numeric values via categorization e.g. TCP - 1, UDP -2 ...
for category in cate_cols:
    labels, unique_values = pd.factorize(df[category])
    df[category] = labels
    category_mapping[category] = dict(zip(set(unique_values), set(labels)))

df.head()

In [ ]:
category_mapping

In [ ]:
for category in cate_cols:
    print(df[category].value_counts())

In [ ]:
df = df[[col for col in df if df[col].nunique() > 1]]# keep columns where there are more than 1 unique values

corr = df.corr()

plt.figure(figsize=(8,5))
sns.heatmap(corr, cmap='Blues')

In [ ]:
X = torch.tensor(df.drop('label',axis=1).values.astype(np.float32))
y = torch.tensor(df['label'].values, dtype=torch.long)
X,y

In [ ]:
y.size()

In [ ]:
X.shape

In [ ]:
def pct_to_val(train_pct, data):
    '''Helper function to make code cleaer.
        changes percentage split into numbers of data.
        INPUTS:
        train_pct: the percentage of training data 
        valid_pct: the percentage of validation data
        data: the dataset
        returns: numbers of data'''
    train_num = int(train_pct/100*len(data))
    valid_num = int(len(data) - train_num)
    return train_num , valid_num

In [ ]:
# OPT::::: 1 but cannot stratify with this strategy
# dataset = TensorDataset(X, y)
# train_num,valid_num = pct_to_val(75, dataset)
# Get traning and validation datasets.
# train_ds,valid_ds = random_split(dataset, [train_num, valid_num])

# OPT::::::2 ## stratify using train_test_split from sklearn
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)
train_ds,valid_ds = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)
# Make a dataloader
BATCH = 1024
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=BATCH, shuffle=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(d, device) for d in data]
    return data.to(device, non_blocking=True)

In [ ]:
class DeviceDL():
    
    def __init__(self, dl, dev):
        self.dl = dl
        self.dev = dev
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.dev)
            
    def __len__(self):
        return len(self.dl)

In [ ]:
train_dl = DeviceDL(train_dl, device)
valid_dl = DeviceDL(valid_dl, device)

In [ ]:
def accuracy(label, preds):
    
    pred_labels = torch.argmax(preds, dim=1)
    # Check how many predictions match the ground truth labels
    correct_predictions = (pred_labels == label).sum() # Cast to int with .item()
    
    # Calculate accuracy
    accuracy = correct_predictions / len(pred_labels)
    return accuracy

In [ ]:
class BaseNet(nn.Module):
    def get_loss(self, batch, loss_fn):
        features,labels = batch
        preds = self(features)
        loss = loss_fn(preds, labels)
        return loss
    
    def validate(self, batch, loss_fn):
        feature, labels = batch
        loss = self.get_loss(batch, loss_fn)
        pred = self(feature)
        
        acc = accuracy(labels, pred)
        return {'valid_loss' : loss , 'valid_acc' : acc}
    
    def average_validation(self, out):
        loss = torch.stack([l['valid_loss'] for l in out]).mean()
        acc = torch.stack([l['valid_acc'] for l in out]).mean()
        return {'valid_loss': loss.item() , 'valid_acc': acc.item()}
    
    def log_epoch(self, e, epoch, res):
        
        print('[{} / {}] epoch/s, training loss is {:.4f} validation loss is {:.4f}, validation accuracy is {:.4f} '\
              .format(e+1,epoch,
                      res['train_loss'],
                      res['valid_loss'],                
                      res['valid_acc']
                     )
             )

In [ ]:
class ClassifierMLP(BaseNet):
    
    def __init__(self, activation, input_dim, hidden_1, hidden_2, hidden_3, out):
        super().__init__()
        self.hidden_1 = nn.Linear(input_dim, hidden_1)
        self.hidden_2 = nn.Linear(hidden_1, hidden_2)
        self.hidden_3 = nn.Linear(hidden_2, hidden_3)
        self.out = nn.Linear(hidden_3, out)
        self.activation = activation
        
    def forward(self, x):
        x1 = self.hidden_1(x)
        x2 = self.activation(x1)
        x3 = self.hidden_2(x2)
        x4 = self.activation(x3)
        x5 = self.hidden_3(x4)
        x6 = self.activation(x5)
        out = self.out(x6)
#         Note in pytorch we don't apply activation in final layer or use softmax because cross entropy
# loss function would do it automatically under the hood, we will rather only use softmax when interpreting
# the results in a human readable form. (not even while testing)
        return out
        
        

In [ ]:
@torch.no_grad()
def valid(model, valid_dl, loss_fn):
    model.eval()
    out = [model.validate(batch, loss_fn) for batch in valid_dl]
    return model.average_validation(out)
    
def fit(model, train_dl, valid_dl, loss_fn, opt, EPOCHS):
    hist = []
    for e in range(EPOCHS):
        model.train()
        train_loss =[]
        for batch in train_dl:
            
            loss = model.get_loss(batch, loss_fn)
            train_loss.append(loss)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        res = valid(model, valid_dl, loss_fn)
        res['train_loss'] = torch.stack(train_loss).mean().item()
        
        model.log_epoch(e, EPOCHS, res)

        hist.append(res)
    return hist

In [ ]:
activation = nn.ReLU()
input_dim = X.shape[1]
output_dim = df['label'].nunique()
hidden_1, hidden_2, hidden_3 = 256, 128, 128


model = ClassifierMLP(activation, input_dim, hidden_1, hidden_2, hidden_3, output_dim)
to_device(model, device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
lr = 0.000001
opt = torch.optim.Adam(model.parameters(), lr=lr)
EPOCHS = 20

In [ ]:
history = fit(model, train_dl, valid_dl, loss_fn, opt, EPOCHS)

In [ ]:
epochs = range(1,len(history)+1)
val_acc_values = [entry['valid_acc'] for entry in history]
test_loss_values = [entry['train_loss'] for entry in history]
val_loss_values = [entry['valid_loss'] for entry in history]

In [ ]:
!mkdir graphs

In [ ]:
plt.style.use('ggplot')
# Create a new figure
plt.figure(figsize=(10, 6))

# Plot validation accuracy
# plt.plot(epochs, val_acc_values, label='Validation Accuracy', marker='o', linestyle='-')
# Plot test loss
plt.plot(epochs, test_loss_values, label='Train Loss', marker='o', linestyle='-')
# Plot validation loss
plt.plot(epochs, val_loss_values, label='Validation Loss', marker='o', linestyle='-')

# Add labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/Training_history_standard.png', dpi=300)

In [ ]:
plt.plot(epochs, val_acc_values, label='Validation Accuracy', marker='o', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Validation Accuracy')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/validation_acc_standard.png', dpi=300)

In [ ]:
model.eval()
y_pred = model(X_test.to(device))
y_preds = torch.argmax(y_pred, axis = 1).to('cpu').detach().numpy()

In [ ]:
!mkdir classification_reports

In [ ]:
category_mapping['label']

In [ ]:
report = classification_report(y_test, y_preds, zero_division=1, target_names=category_mapping['label'].keys(), output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(4)
# Save the DataFrame as a CSV file
report_df.to_csv('classification_reports/classification_standard.csv', index=True)

print(report_df)

In [ ]:
del report_df

In [ ]:
confusion = confusion_matrix(y_test, y_preds)

categories = category_mapping['label'].keys()
plt.figure(figsize=(25,20))

sns.heatmap(confusion,
            annot=True,
            cmap='Blues', 
            xticklabels = categories,
            yticklabels = categories,
            vmin=0, vmax=10000
            )
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('graphs/confusion_standard.png', dpi=300)

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())

print(dict(zip(category_mapping['label'].keys(),class_weights)))
print(init_df['label'].value_counts())

class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
model_weighted = ClassifierMLP(activation, input_dim, hidden_1, hidden_2, hidden_3, output_dim)
to_device(model_weighted, device)

In [ ]:
loss_fn = nn.CrossEntropyLoss(class_weights)
lr = 0.000001
opt = torch.optim.Adam(model_weighted.parameters(), lr=lr)

In [ ]:
history_weighted = fit(model_weighted, train_dl, valid_dl, loss_fn, opt, EPOCHS)

In [ ]:
val_acc_weighted_values = [entry['valid_acc'] for entry in history_weighted]
test_loss_weighted_values = [entry['train_loss'] for entry in history_weighted]
val_loss_weighted_values = [entry['valid_loss'] for entry in history_weighted]

In [ ]:
plt.style.use('ggplot')
# Create a new figure
plt.figure(figsize=(10, 8))

# Plot validation accuracy
# plt.plot(epochs, val_acc_values, label='Validation Accuracy', marker='o', linestyle='-')
# Plot test loss
plt.plot(epochs, test_loss_weighted_values, label='Train Loss', marker='o', linestyle='-')
# Plot validation loss
plt.plot(epochs, val_loss_weighted_values, label='Validation Loss', marker='o', linestyle='-')

# Add labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/training_history_weighted.png', dpi=300)

In [ ]:
plt.plot(epochs, val_acc_weighted_values, label='Validation Accuracy', marker='o', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Validation Accuracy')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/validation_acc_weighted.png', dpi=300)

In [ ]:
model_weighted.eval()

In [ ]:
y_pred = model_weighted(X_test.to(device))
y_preds = torch.argmax(y_pred, axis = 1).to('cpu').detach().numpy()

In [ ]:
report = classification_report(y_test, y_preds, zero_division=1, target_names=category_mapping['label'].keys(), output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(4)
# Save the DataFrame as a CSV file
report_df.to_csv('classification_reports/classification_weighted.csv', index=True)

print(report_df)

In [ ]:
confusion = confusion_matrix(y_test, y_preds)

categories = category_mapping['label'].keys()
plt.figure(figsize=(25,20))

sns.heatmap(confusion,
            annot=True,
            cmap='Blues', 
            xticklabels = categories,
            yticklabels = categories,
            vmin=0, vmax=10000
            )
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('graphs/confusion_weighted.png', dpi=300)

In [ ]:
!mkdir models

In [ ]:
torch.save(model.state_dict(), 'models/model.pth')
torch.save(model_weighted.state_dict(), 'models/model_weighted.pth')

In [ ]:
# free-up some ram
del df
del model
del model_weighted
del train_dl
del valid_dl
del X_train, X_test, y_train, y_test 
del train_ds, valid_ds

In [ ]:
init_df['label'].value_counts()

In [ ]:
label_mapping = {
    'back.': 1, 'land.': 1, 'neptune.': 1, 'pod.': 1, 'smurf.': 1, 'teardrop.': 1, 'apache2.': 1,
    'udpstorm.': 1, 'processtable.': 1, 'worm.': 1,
    
    'satan.': 2, 'ipsweep.': 2, 'nmap.': 2, 'portsweep.': 2, 'mscan.': 2, 'saint.': 2,
    
    'guess_passwd.': 3, 'ftp_write.': 3, 'imap.': 3, 'phf.': 3, 'multihop.': 3, 'warezmaster.': 3,
    'warezclient.': 3, 'spy.': 3, 'xlock.': 3, 'xsnoop.': 3, 'snmpguess.': 3, 'snmpgetattack.': 3,
    'httptunnel.': 3, 'sendmail.': 3, 'named.': 3, 'mailbomb.': 3,
    
    'buffer_overflow.': 3, 'loadmodule.': 3, 'rootkit.': 3, 'perl.': 3, 'sqlattack.': 3,
    'xterm.': 3, 'ps.': 3,
    
    'normal.': 0
}

init_df['label'] = init_df['label'].replace(label_mapping)


In [ ]:
init_df['label'].value_counts()

In [ ]:
category_mapping_truncated = {}

In [ ]:
cate_cols.remove('label')

In [ ]:
for category in cate_cols:
    labels, unique_values = pd.factorize(init_df[category])
    init_df[category] = labels
    category_mapping_truncated[category] = dict(zip(set(unique_values), set(labels)))

In [ ]:
X = torch.tensor(init_df.drop('label',axis=1).values.astype(np.float32))
y = torch.tensor(init_df['label'].values, dtype=torch.long)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)
train_ds,valid_ds = TensorDataset(X_train, y_train), TensorDataset(X_test, y_test)
# Make a dataloader
BATCH = 1024
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=BATCH, shuffle=True)

train_dl = DeviceDL(train_dl, device)
valid_dl = DeviceDL(valid_dl, device)

In [ ]:
class TruncatedClassifierMLP(BaseNet):
    
    def __init__(self, activation, input_dim, hidden_1, hidden_2, out):
        super().__init__()
        self.hidden_1 = nn.Linear(input_dim, hidden_1)
        self.hidden_2 = nn.Linear(hidden_1, hidden_2)
        self.out = nn.Linear(hidden_2, out)
        self.activation = activation
        
    def forward(self, x):
        x1 = self.hidden_1(x)
        x2 = self.activation(x1)
        x3 = self.hidden_2(x2)
        x4 = self.activation(x3)
        out = self.out(x4)
        
        return out

In [ ]:
activation = nn.ReLU()
input_dim = X.shape[1]
output_dim = init_df['label'].nunique()
hidden_1, hidden_2 = 128, 64


truncated_model = TruncatedClassifierMLP(activation, input_dim, hidden_1, hidden_2, output_dim)
to_device(truncated_model, device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
lr = 0.000001
opt = torch.optim.Adam(truncated_model.parameters(), lr=lr)

In [ ]:
history_truncated = fit(truncated_model, train_dl, valid_dl, loss_fn, opt, EPOCHS)

In [ ]:
val_acc_values = [entry['valid_acc'] for entry in history_truncated]
test_loss_values = [entry['train_loss'] for entry in history_truncated]
val_loss_values = [entry['valid_loss'] for entry in history_truncated]

In [ ]:
plt.style.use('ggplot')
# Create a new figure
plt.figure(figsize=(10, 8))

# Plot validation accuracy
# plt.plot(epochs, val_acc_values, label='Validation Accuracy', marker='o', linestyle='-')
# Plot test loss
plt.plot(epochs, test_loss_values, label='Train Loss', marker='o', linestyle='-')
# Plot validation loss
plt.plot(epochs, val_loss_values, label='Validation Loss', marker='o', linestyle='-')

# Add labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/training_history_truncated_four_class.png', dpi=300)

In [ ]:
plt.plot(epochs, val_acc_values, label='Validation Accuracy', marker='o', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Validation Accuracy')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/validation_acc_truncated_four_class.png', dpi=300)

In [ ]:
truncated_model.eval()

y_pred = truncated_model(X_test.to(device))
y_preds = torch.argmax(y_pred, axis = 1).to('cpu').detach().numpy()

In [ ]:
category_mapping_truncated = ['Normal', 'probe','DOS', 'Unauthorized Access']

In [ ]:
report = classification_report(y_test, y_preds, zero_division=1, target_names=category_mapping_truncated, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(4)
# Save the DataFrame as a CSV file
report_df.to_csv('classification_reports/classification_truncated_four_class.csv', index=True)

print(report_df)

In [ ]:
confusion = confusion_matrix(y_test, y_preds)

categories = category_mapping_truncated
# plt.figure(figsize=(25,20))

sns.heatmap(confusion,
            annot=True,
            cmap='Blues', 
            xticklabels = categories,
            yticklabels = categories,
            vmin=0, vmax=10000
            )
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('graphs/confusion_truncated_four_class.png', dpi=300)

In [ ]:
torch.save(truncated_model.state_dict(), 'models/model_truncated_four_class.pth')
del truncated_model

In [ ]:
truncated_weighted_model = TruncatedClassifierMLP(activation, input_dim, hidden_1, hidden_2, output_dim)
to_device(truncated_weighted_model, device)

In [ ]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y.numpy())

print(dict(zip(init_df['label'].unique(),class_weights)))
print(init_df['label'].value_counts())

class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss(class_weights)
lr = 0.000001
opt = torch.optim.Adam(truncated_weighted_model.parameters(), lr=lr)

In [ ]:
history_weighted_truncated = fit(truncated_weighted_model, train_dl, valid_dl, loss_fn, opt, EPOCHS)

In [ ]:
val_acc_weighted_values = [entry['valid_acc'] for entry in history_weighted_truncated]
test_loss_weighted_values = [entry['train_loss'] for entry in history_weighted_truncated]
val_loss_weighted_values = [entry['valid_loss'] for entry in history_weighted_truncated]

In [ ]:
plt.style.use('ggplot')
# Create a new figure
plt.figure(figsize=(10, 8))

# Plot validation accuracy
# plt.plot(epochs, val_acc_values, label='Validation Accuracy', marker='o', linestyle='-')
# Plot test loss
plt.plot(epochs, test_loss_weighted_values, label='Train Loss', marker='o', linestyle='-')
# Plot validation loss
plt.plot(epochs, val_loss_weighted_values, label='Validation Loss', marker='o', linestyle='-')

# Add labels and legend
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/training_history_truncated_weighted_four_class.png', dpi=300)

In [ ]:
plt.plot(epochs, val_acc_weighted_values, label='Validation Accuracy', marker='o', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Value')
plt.title('Validation Accuracy')
plt.legend()

# Show the plot
plt.grid(True)
plt.savefig('graphs/validation_acc_truncated_weighted_four_class.png', dpi=300)

In [ ]:
truncated_weighted_model.eval()

y_pred = truncated_weighted_model(X_test.to(device))
y_preds = torch.argmax(y_pred, axis = 1).to('cpu').detach().numpy()

In [ ]:
report = classification_report(y_test, y_preds, zero_division=1, target_names=category_mapping_truncated, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(4)
# Save the DataFrame as a CSV file
report_df.to_csv('classification_reports/classification_truncated_weighted_four_class.csv', index=True)

print(report_df)

In [ ]:
confusion = confusion_matrix(y_test, y_preds)

categories = category_mapping_truncated
# plt.figure(figsize=(25,20))

sns.heatmap(confusion,
            annot=True,
            cmap='Blues', 
            xticklabels = categories,
            yticklabels = categories,
            vmin=0, vmax=10000
            )
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.savefig('graphs/confusion_truncated_weighted_four_class.png', dpi=300)

In [ ]:
torch.save(truncated_weighted_model.state_dict(), 'models/model_truncated_weighted_four_class.pth')

In [ ]:
!tar czf output_folders.tar.gz *